In this notebook we begin implementing sparse bayesian learning.

In [1]:
# General imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from deepmod_l1.analytical import theta_analytical

#Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# DeepMoD stuff
from deepymod_torch.DeepMod import DeepMod, build_network
from deepymod_torch.library_functions import library_basic
from deepymod_torch.utilities import create_deriv_data
from deepymod_torch.output import progress

# Remainder imports
from os import listdir, path, getcwd

# Setting cuda
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Defining output folder
output_folder = getcwd()

%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

# Making library

In [3]:
D = 0.5
a = 0.25

x = np.linspace(-5, 5, 50, dtype=np.float32)
t = np.linspace(0, 5, 50, dtype=np.float32)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
    
# Analytical
time_deriv, theta_noiseless = theta_analytical(x_grid, t_grid, D, a)

And performing lst-sq we get:

In [ ]:
xi_base = np.linalg.lstsq(theta, time_deriv, rcond=None)[0].squeeze()

In [ ]:
xi_base

Now let's add some noise:

In [ ]:
time_deriv = time_deriv + np.random.normal(scale=0.1, size=time_deriv.shape)

In [ ]:
np.linalg.lstsq(theta, time_deriv, rcond=None)[0].squeeze()

# Sparse bayesian learning

In [ ]:
# Setting sigma to reasonable value
sigma2 = np.var(time_deriv) * 0.1

In [ ]:
# Finding best initial vector 
projection = np.concatenate([((theta[:, idx:idx+1].T @ time_deriv).T @ (theta[:, idx:idx+1].T @ time_deriv))/(theta[:, idx:idx+1].T @ theta[:, idx:idx+1]) for idx in np.arange(theta.shape[1])])
start_idx = np.argmax(projection)

# Initializing alphas
a = np.ones((theta.shape[1], 1)) * np.inf
a[start_idx] = theta[:, start_idx:start_idx+1].T @ theta[:, start_idx:start_idx+1] / (projection[start_idx] - sigma2)

In [ ]:
projection

In [ ]:
start_idx

In [ ]:
a

In [ ]:
# Calculating sigma and mu
current_matrix = theta[:, [start_idx]]
sigma = np.linalg.inv(a[start_idx][:, None] + 1/sigma2 * current_matrix.T @ current_matrix)

mu = 1/sigma2 * sigma @ current_matrix.T @ time_deriv

In [ ]:
sigma

In [ ]:
mu

In [ ]:
# calculating Sm and Qm

B = 1/sigma2 * np.eye(current_matrix.shape[0])
precalc = B @ current_matrix @ sigma @ current_matrix.T @ B

Sm = np.concatenate([theta[:, idx:idx+1].T @ B @ theta[:, idx:idx+1] - theta[:, idx:idx+1].T @ precalc @ theta[:, idx:idx+1] for idx in np.arange(theta.shape[1])])
Qm = np.concatenate([theta[:, idx:idx+1].T @ B @ time_deriv - theta[:, idx:idx+1].T @ precalc @ time_deriv for idx in np.arange(theta.shape[1])])

sm = Sm/(1 - Sm/a)
qm = Qm/(1 - Sm/a)

In [ ]:
sm

In [ ]:
qm

In [ ]:
# Randomly pick basis vector to improve and calculate theta_i
idx = np.random.choice(theta.shape[1])
t = (qm[idx]**2 - sm[idx]).item()

In [ ]:
idx

In [ ]:
t

In [ ]:
if (t > 0) & (a[idx].item() != np.inf):
    print('restimate')
elif (t > 0) & (a[idx].item() == np.inf):
    print('add and update')
elif (t < 0) & (a[idx].item() != np.inf):
    print('delete that shit')
elif (t < 0) & (a[idx].item() == np.inf):
    print('dont add the fucker')

In [ ]:
# Doing it all in a loop

for iteration in np.arange(100):
    idx = np.random.choice(theta.shape[1])
    t = (qm[idx]**2 - sm[idx]).item()
    
    # Decidiing what to do 
    if (t > 0) & (a[idx].item() != np.inf):
        a[idx] = sm[idx]**2/(qm[idx]**2-sm[idx]) # reestimating
    elif (t > 0) & (a[idx].item() == np.inf):
        a[idx] = sm[idx]**2/(qm[idx]**2-sm[idx]) # adding alpha
        current_matrix = np.concatenate((current_matrix, theta[:, idx:idx+1]), axis=1) # adding to design matrix
    elif (t < 0) & (a[idx].item() != np.inf):
        a[idx] = np.inf #removing alpha
        current_matrix = np.delete(current_matrix, idx, axis=0) #removing from design matrix
    elif (t < 0) & (a[idx].item() == np.inf):
        pass #do nothing;
    
    # Matrices
    sigma = np.linalg.inv(a[a!=np.inf][:, None] + 1/sigma2 * current_matrix.T @ current_matrix)
    mu = 1/sigma2 * sigma @ current_matrix.T @ time_deriv

    B = 1/sigma2 * np.eye(current_matrix.shape[0])
    precalc = B @ current_matrix @ sigma @ current_matrix.T @ B

    Sm = np.concatenate([theta[:, idx:idx+1].T @ B @ theta[:, idx:idx+1] - theta[:, idx:idx+1].T @ precalc @ theta[:, idx:idx+1] for idx in np.arange(theta.shape[1])])
    Qm = np.concatenate([theta[:, idx:idx+1].T @ B @ time_deriv - theta[:, idx:idx+1].T @ precalc @ time_deriv for idx in np.arange(theta.shape[1])])

    sm = Sm/(1 - Sm/a)
    qm = Qm/(1 - Sm/a)
    
    if iteration % 10 == 0:
        print(f'done with {iteration}')

In [ ]:
current_matrix.shape

In [ ]:
a

In [ ]:
a

The posterior is simply mu:

In [ ]:
mu

In [ ]:
sigma

# Rewriting variable names so we get working version

In [4]:
noise = np.var(time_deriv) * 0.01
normalization = np.linalg.norm(theta_noiseless, axis=0)
theta = theta_noiseless #/ normalization
t = time_deriv #+ np.random.normal(scale= np.sqrt(noise), size=time_deriv.shape)

In [20]:
# Initialisation
sigma2 = np.var(time_deriv) * 0.1

# Finding best initial vector 
projection = np.concatenate([((phi_i[:, None].T @ t).T @ (phi_i[:, None].T @ t)) / (phi_i[:, None].T @ phi_i[:, None]) for phi_i in theta.T])
start_idx = np.argmax(projection)

# Initializing alphas
a = np.ones((theta.shape[1], 1)) * np.inf
a[start_idx] = theta[:, start_idx:start_idx+1].T @ theta[:, start_idx:start_idx+1] / (projection[start_idx] - sigma2)
Phi = theta[:, [start_idx]]

# Calculating sigma and mu
Sigma = np.linalg.inv(a[a!=np.inf] * np.eye(Phi.shape[1]) + 1/sigma2 * Phi.T @ Phi)
mu = 1/sigma2 * Sigma @ Phi.T @ t

# Calculating s and m
B = 1/sigma2 * np.eye(Phi.shape[0])
precalc = B @ Phi @ Sigma @ Phi.T @ B

Sm = np.concatenate([phi_i[:, None].T @ B @ phi_i[:, None] - phi_i[:, None].T @ precalc @ phi_i[:, None] for phi_i in theta.T])
Qm = np.concatenate([phi_i[:, None].T @ B @ t - phi_i[:, None].T @ precalc @ t for phi_i in theta.T])

sm = Sm/(1 - Sm/a)
qm = Qm/(1 - Sm/a)

In [6]:
1/sigma2

81.92196504713522

In [7]:
start_idx

2

In [8]:
mu

array([[0.49997995]])

In [23]:
qm**2 - sm

array([[-204804.55585766],
       [  -6354.7387566 ],
       [ 584658.81763924],
       [  -4313.6915979 ],
       [  -3806.81492184],
       [ -29071.88096104],
       [   -749.24529638],
       [  -5297.42479995],
       [ -86201.60296754]])

In [ ]:
# Doing it all in a loop
for iteration in np.arange(10):
    idx = np.random.choice(theta.shape[1])
    phi_i = theta[:, idx:idx+1]
    theta_i = (qm[idx, 0]**2 - sm[idx, 0])
    print(theta_i)
    sigma2 = (t - Phi @ mu).T @ (t - Phi @ mu) / (Phi.shape[0] - Phi.shape[1] + np.sum(a[a!=np.inf] * np.diag(Sigma)))
    
    # Decididing what to do 
    if (theta_i > 0) & (a[idx, 0] != np.inf):
        a[idx, 0] = sm[idx, 0]**2 / theta_i #reestimating
    elif (theta_i > 0) & (a[idx, 0] == np.inf):
        a[idx, 0] = sm[idx, 0]**2 / theta_i # adding alpha
        Phi = theta[:, a[:, 0] != np.inf] # we rebuild completely because we need to preserve order
    elif (theta_i< 0) & (a[idx, 0] != np.inf):
        a[idx, 0] = np.inf #removing alpha
        Phi = theta[:, a[:, 0] != np.inf] # we rebuild completely because we need to preserve order
    elif (theta_i < 0) & (a[idx, 0] == np.inf):
        pass #do nothing;
    
    # Calculating sigma and mu
    Sigma = np.linalg.inv(a[a!=np.inf] * np.eye(Phi.shape[1]) + 1/sigma2 * Phi.T @ Phi)
    mu = 1/sigma2 * Sigma @ Phi.T @ t
    
    # Matrices
    B = 1/sigma2 * np.eye(Phi.shape[0])
    precalc = B @ Phi @ Sigma @ Phi.T @ B

    Sm = np.concatenate([phi_i[:, None].T @ B @ phi_i[:, None] - phi_i[:, None].T @ precalc @ phi_i[:, None] for phi_i in theta.T])
    Qm = np.concatenate([phi_i[:, None].T @ B @ t - phi_i[:, None].T @ precalc @ t for phi_i in theta.T])

    sm = Sm/(1 - Sm/a)
    qm = Qm/(1 - Sm/a)
    
    if iteration % 25 == 0:
        print(f'done with {iteration}')

In [ ]:
Phi.shape

In [ ]:
Sigma.shape

it works; let's add a little noise

In [ ]:
mu 

In [ ]:
a

So although it doesn't work, clearly it's much more sure about the terms we require...

Still works! Now we need to figure out how to estimate the noise as well...

In [ ]:
sigma2

In [ ]:
noise

Let's look at the normalized ones:

In [ ]:
mu[:, 0] / normalization[a[:, 0] != np.inf]

In [ ]:
a[:, 0] / normalization

So we now have a basic version working, the next step is to precalculate some quantities to figure out which basis to add, instead of randomly assigning.

# Adding basis vector selection:

In [175]:
D = 0.5
a = 0.25

x = np.linspace(-5, 5, 50, dtype=np.float32)
t = np.linspace(0, 5, 50, dtype=np.float32)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
    
# Analytical
time_deriv, theta_noiseless = theta_analytical(x_grid, t_grid, D, a)

In [176]:
noise = np.var(time_deriv) * 0.00
normalization = np.linalg.norm(theta_noiseless, axis=0)
theta = theta_noiseless / normalization
t = time_deriv + np.random.normal(scale= np.sqrt(noise), size=time_deriv.shape)

In [177]:
# Initialisation
sigma2 = np.var(time_deriv) * 0.1

# Finding best initial vector 
projection = np.concatenate([((phi_i[:, None].T @ t).T @ (phi_i[:, None].T @ t)) / (phi_i[:, None].T @ phi_i[:, None]) for phi_i in theta.T])
start_idx = np.argmax(projection)

# Initializing alphas
a = np.ones((theta.shape[1], 1)) * np.inf
a[start_idx] = theta[:, start_idx:start_idx+1].T @ theta[:, start_idx:start_idx+1] / (projection[start_idx] - sigma2)
Phi = theta[:, [start_idx]]

# Calculating sigma and mu
Sigma = np.linalg.inv(a[a!=np.inf] * np.eye(Phi.shape[1]) + 1/sigma2 * Phi.T @ Phi)
mu = 1/sigma2 * Sigma @ Phi.T @ t

# Calculating s and m
B = 1/sigma2 * np.eye(Phi.shape[0])
precalc = B @ Phi @ Sigma @ Phi.T @ B

Sm = np.concatenate([phi_i[:, None].T @ B @ phi_i[:, None] - phi_i[:, None].T @ precalc @ phi_i[:, None] for phi_i in theta.T])
Qm = np.concatenate([phi_i[:, None].T @ B @ t - phi_i[:, None].T @ precalc @ t for phi_i in theta.T])

sm = Sm/(1 - Sm/a)
qm = Qm/(1 - Sm/a)

In [178]:
# Doing itpossible_update in a loop
for iteration in np.arange(101):
    # Choosing basis function
    basis_idx = a != np.inf
    set_idx = a == np.inf
    
    add_basis = (Qm**2 - Sm)/Sm + np.log(Sm/Qm**2)
    del_basis = Qm**2/(Sm - a) - np.log(1-Sm/a)
    a_new = sm**2/(qm**2 - sm)
    redo_basis = Qm**2/(Sm + (1/a_new-1/a)**-1) - np.log(1 + Sm * (1/a_new-1/a))
    
    #Making everything into nice matrix
    add_basis[basis_idx] = np.nan
    del_basis[set_idx] = np.nan
    redo_basis[set_idx] = np.nan
    
    # Deciding update
    possible_update = np.concatenate((add_basis, redo_basis, del_basis), axis=1)
    idx, opt = np.unravel_index(np.nanargmax(possible_update), possible_update.shape)
    
    # Extracting update
    phi_i = theta[:, idx:idx+1]
    theta_i = (qm[idx, 0]**2 - sm[idx, 0])
    
    sigma2 = (t - Phi @ mu).T @ (t - Phi @ mu) / (Phi.shape[0] - Phi.shape[1] + np.sum(a[a!=np.inf] * np.diag(Sigma)))

    # Decididing what to do 
    if (theta_i > 0) & (a[idx, 0] != np.inf):
        a[idx, 0] = sm[idx, 0]**2 / theta_i #reestimating
    elif (theta_i > 0) & (a[idx, 0] == np.inf):
        a[idx, 0] = sm[idx, 0]**2 / theta_i # adding alpha
        Phi = theta[:, a[:, 0] != np.inf] # we rebuild completely because we need to preserve order
    elif (theta_i< 0) & (a[idx, 0] != np.inf):
        a[idx, 0] = np.inf #removing alpha
        Phi = theta[:, a[:, 0] != np.inf] # we rebuild completely because we need to preserve order
    elif (theta_i < 0) & (a[idx, 0] == np.inf):
        pass #do nothing;
    
    # Calculating sigma and mu
    Sigma = np.linalg.inv(a[a!=np.inf] * np.eye(Phi.shape[1]) + 1/sigma2 * Phi.T @ Phi)
    mu = 1/sigma2 * Sigma @ Phi.T @ t
    
    # Matrices
    B = 1/sigma2 * np.eye(Phi.shape[0])
    precalc = B @ Phi @ Sigma @ Phi.T @ B

    Sm = np.concatenate([phi_i[:, None].T @ B @ phi_i[:, None] - phi_i[:, None].T @ precalc @ phi_i[:, None] for phi_i in theta.T])
    Qm = np.concatenate([phi_i[:, None].T @ B @ t - phi_i[:, None].T @ precalc @ t for phi_i in theta.T])

    sm = Sm/(1 - Sm/a)
    qm = Qm/(1 - Sm/a)
    
    if iteration % 10 == 0:
        print(f'done with {iteration}')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


done with 0


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in log
  # Remove the CWD from sys.path while we load stuff.


done with 10


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in add
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in multiply
  # Remove the CWD from sys.path while we load stuff.


done with 20
done with 30
done with 40


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


done with 50
done with 60
done with 70
done with 80
done with 90
done with 100


In [166]:
mu[:, 0] / normalization[a[:, 0] != np.inf]

array([ 6.64602693e-10,  4.99999981e-01,  2.93714540e-08, -1.82310386e-08])

In [167]:
sigma2

array([[2.58212576e-17]])

In [168]:
a

array([[8.82493786e+14],
       [           inf],
       [3.21683096e-05],
       [           inf],
       [           inf],
       [5.98200943e+11],
       [3.12306011e+14],
       [           inf],
       [           inf]])

In [169]:
possible_update

array([[            nan,  3.07273070e-07, -3.34435727e+01],
       [ 3.33233527e+01,             nan,             nan],
       [            nan,  9.20587130e+03,             nan],
       [ 7.98906668e-01,             nan,             nan],
       [ 3.21061385e+01,             nan,             nan],
       [            nan,  3.58971021e-03, -8.92244935e+03],
       [            nan,  2.36815763e-01, -7.43219661e+01],
       [ 3.04807816e+01,             nan,             nan],
       [ 4.71300605e+00,             nan,             nan]])

# Implementing convergence

In [ ]:
D = 0.5
a = 0.25

x = np.linspace(-5, 5, 50, dtype=np.float32)
t = np.linspace(0, 5, 50, dtype=np.float32)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
    
# Analytical
time_deriv, theta_noiseless = theta_analytical(x_grid, t_grid, D, a)

In [ ]:
noise = np.var(time_deriv) * 0.01
normalization = np.linalg.norm(theta_noiseless, axis=0)
theta = theta_noiseless / normalization
t = time_deriv + np.random.normal(scale= np.sqrt(noise), size=time_deriv.shape)

In [ ]:
# Initialisation
sigma2 = np.var(time_deriv) * 0.1

# Finding best initial vector 
projection = np.concatenate([((phi_i[:, None].T @ t).T @ (phi_i[:, None].T @ t)) / (phi_i[:, None].T @ phi_i[:, None]) for phi_i in theta.T])
start_idx = np.argmax(projection)

# Initializing alphas
a = np.ones((theta.shape[1], 1)) * np.inf
a[start_idx] = theta[:, start_idx:start_idx+1].T @ theta[:, start_idx:start_idx+1] / (projection[start_idx] - sigma2)
Phi = theta[:, [start_idx]]

# Calculating sigma and mu
Sigma = np.linalg.inv(a[a!=np.inf] * np.eye(Phi.shape[1]) + 1/sigma2 * Phi.T @ Phi)
mu = 1/sigma2 * Sigma @ Phi.T @ t

# Calculating s and m
B = 1/sigma2 * np.eye(Phi.shape[0])
precalc = B @ Phi @ Sigma @ Phi.T @ B

Sm = np.concatenate([phi_i[:, None].T @ B @ phi_i[:, None] - phi_i[:, None].T @ precalc @ phi_i[:, None] for phi_i in theta.T])
Qm = np.concatenate([phi_i[:, None].T @ B @ t - phi_i[:, None].T @ precalc @ t for phi_i in theta.T])

sm = Sm/(1 - Sm/a)
qm = Qm/(1 - Sm/a)

In [ ]:
# Doing itpossible_update in a loop
converged = False
for iteration in np.arange(500):#while converged == False:
    idx = np.random.choice(theta.shape[1])
    phi_i = theta[:, idx:idx+1]
    theta_i = (qm[idx, 0]**2 - sm[idx, 0])
    
    sigma2 = (t - Phi @ mu).T @ (t - Phi @ mu) / (Phi.shape[0] - Phi.shape[1] + np.sum(a[a!=np.inf] * np.diag(Sigma)))
    
    # Decididing what to do 
    if (theta_i > 0) & (a[idx, 0] != np.inf):
        a[idx, 0] = sm[idx, 0]**2 / theta_i #reestimating
    elif (theta_i > 0) & (a[idx, 0] == np.inf):
        a[idx, 0] = sm[idx, 0]**2 / theta_i # adding alpha
        Phi = theta[:, a[:, 0] != np.inf] # we rebuild completely because we need to preserve order
    elif (theta_i< 0) & (a[idx, 0] != np.inf):
        a[idx, 0] = np.inf #removing alpha
        Phi = theta[:, a[:, 0] != np.inf] # we rebuild completely because we need to preserve order
    elif (theta_i < 0) & (a[idx, 0] == np.inf):
        pass #do nothing;
    
    # Calculating sigma and mu
    Sigma = np.linalg.inv(a[a!=np.inf] * np.eye(Phi.shape[1]) + 1/sigma2 * Phi.T @ Phi)
    mu = 1/sigma2 * Sigma @ Phi.T @ t
    
    # Matrices
    B = 1/sigma2 * np.eye(Phi.shape[0])
    precalc = B @ Phi @ Sigma @ Phi.T @ B

    Sm = np.concatenate([phi_i[:, None].T @ B @ phi_i[:, None] - phi_i[:, None].T @ precalc @ phi_i[:, None] for phi_i in theta.T])
    Qm = np.concatenate([phi_i[:, None].T @ B @ t - phi_i[:, None].T @ precalc @ t for phi_i in theta.T])

    sm = Sm/(1 - Sm/a)
    qm = Qm/(1 - Sm/a)
    
    if iteration % 10 == 0:
        print(f'done with {iteration}')
    
    # Converging
    dt = qm**2 - sm
    delta_a= sm**2/dt - a
    converged = np.max(np.abs(delta_a[dt > 0])) < 10**-6
    
    if converged: 
        print('converged')
        print(a)

In [69]:
a

array([[1.11152981e+02],
       [           inf],
       [3.27709598e-03],
       [2.52544372e+02],
       [           inf],
       [           inf],
       [           inf],
       [           inf],
       [           inf]])

In [70]:
dt = qm**2 - sm
dt

array([[ 2.96920850e+03],
       [-8.36430283e+02],
       [ 1.91866491e+08],
       [ 9.10250409e+02],
       [-8.66195978e+02],
       [-2.54518525e+01],
       [-2.15354796e+02],
       [-8.63892971e+02],
       [-2.61490094e+01]])

In [71]:
delta_a= sm**2/dt - a

In [72]:
np.max(np.abs(delta_a[dt > 0])) < 10**-6

True

In [73]:
delta_a

array([[ 2.04920525e-11],
       [           -inf],
       [-7.34682280e-13],
       [ 1.97672989e-10],
       [           -inf],
       [           -inf],
       [           -inf],
       [           -inf],
       [           -inf]])

In [74]:
add_basis = (Qm**2 - Sm)/Sm + np.log(Sm/Qm**2)
del_basis = Qm**2/(Sm - a) - np.log(1-Sm/a)

a_new = sm**2/(qm**2 - sm)
redo_basis = Qm**2/(Sm + (1/a_new-1/a)**-1) - np.log(1 + Sm * (1/a_new-1/a))

In [75]:
add_basis

array([[-2.66478123e-17],
       [ 2.19987039e+00],
       [-3.00432611e-17],
       [-1.12409877e-17],
       [ 3.78010886e+00],
       [ 1.21379986e-02],
       [ 1.48708439e+00],
       [ 3.51138727e+00],
       [ 5.87217172e-03]])

In [76]:
del_basis

array([[-3.34899712e+00],
       [-0.00000000e+00],
       [-2.41953961e+05],
       [-8.34309480e-01],
       [-0.00000000e+00],
       [-0.00000000e+00],
       [-0.00000000e+00],
       [-0.00000000e+00],
       [-0.00000000e+00]])

In [77]:
redo_basis

array([[-1.99462142e-17],
       [ 2.19987039e+00],
       [ 5.44606788e-17],
       [ 3.31201958e-17],
       [ 3.78010886e+00],
       [ 1.21379986e-02],
       [ 1.48708439e+00],
       [ 3.51138727e+00],
       [ 5.87217172e-03]])